In [1]:
import json
import os


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler


from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest, AdaBoostClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score


from tensorflow import keras
from tensorflow.keras import layers, models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

2025-10-26 13:49:35.051177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761486575.280918      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761486575.348767      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class AnomalyMethods:

    def __init__(self):
        self.name = None
        self.age = None
    
    def using_knn(self, data, model, n_neighbors=3, percentile=95, predictions=True):
        model.fit(data)

        # Compute distances
        distances, indices = model.kneighbors(data, n_neighbors=n_neighbors)

        # Compute anomaly scores
        anomaly_scores = distances.mean(axis=1)

        # Set threshold
        threshold = np.percentile(anomaly_scores, percentile)

        labels=None
        if predictions:
            labels = np.where(anomaly_scores > threshold, -1, 1) # -1: anomaly, 1: inlier
            
        return model, labels
    
    def using_pca(self, data, model, variance_explained=0.95, predictions=True):
        pca = PCA(n_components=variance_explained)  # Retain 95% variance if variance_explained=0.95
        X_pca = pca.fit_transform(data)

        # Reconstruct data
        projected_data = pca.inverse_transform(X_pca)

        # Compute reconstruction error
        reconstruction_error = np.mean((data - projected_data) ** 2, axis=1)

        # Anomaly threshold (95th percentile)
        threshold = np.percentile(reconstruction_error, 95)

        labels = None
        if predictions:
            labels = np.where(reconstruction_error > threshold, -1, 1) # -1: anomaly, 1: inlier
        
        return pca, labels
    
    def using_adaboost(self, X_train, Y_train, X_test, model, predictions=True):
        model.fit(X_train, Y_train)

        labels = None
        if predictions:
            lables = model.predict(X_test)
            y_pred_proba = model.predict_proba(X_test)[:, 1] # Probability of anomaly class
        
        return model, labels

In [3]:
def merge_csv_files_recursive(directory_path):
    merged_data = pd.DataFrame()

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                merged_data = pd.concat([merged_data, df], ignore_index=True)

    return merged_data

In [4]:
dataset_path = '/kaggle/input/anomaly-detection-falling-people-events/data/train/'
fall_dataset = merge_csv_files_recursive(dataset_path)

test_path = '/kaggle/input/anomaly-detection-falling-people-events/data/test/'
test_df = merge_csv_files_recursive(test_path)

In [5]:
fall_dataset.head()

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,anomaly
0,4.451597,10.051419,8.836623,0.0,0.0,1.0,0.0,0.0
1,4.395443,10.054421,8.578773,0.0,1.0,0.0,0.0,0.0
2,4.395812,10.043989,8.601673,1.0,0.0,0.0,0.0,0.0
3,4.446850,10.084956,8.429971,0.0,0.0,1.0,0.0,0.0
4,4.388742,10.094799,8.161707,0.0,1.0,0.0,0.0,0.0


In [6]:
fall_dataset.shape

(134229, 8)

In [7]:
fall_dataset.anomaly.value_counts()

anomaly
0.0    127656
1.0      6573
Name: count, dtype: int64

In [8]:
fall_dataset.columns

Index(['x', 'y', 'z', '010-000-024-033', '010-000-030-096', '020-000-032-221',
       '020-000-033-111', 'anomaly'],
      dtype='object')

In [9]:
columns_to_scale = ['x', 'y', 'z']

feature_scale = fall_dataset[columns_to_scale]

test_scale = test_df[columns_to_scale]

fall_dataset['label'] = fall_dataset['anomaly'].replace({0:1, 1:-1})
test_df['label'] = test_df['anomaly'].replace({0:1, 1:-1})

scaler = StandardScaler()
scaled_features = scaler.fit_transform(fall_dataset[['x', 'y', 'z']])
saceld_test = scaler.transform(test_df[['x', 'y', 'z']])

scaled_df = pd.DataFrame(scaled_features, columns=columns_to_scale, index=fall_dataset.index)
saceld_test_df = pd.DataFrame(saceld_test, columns=columns_to_scale, index=test_df.index)

df_final = pd.concat([scaled_df, fall_dataset.drop(columns=columns_to_scale+['anomaly'])], axis=1)
final_test_df = pd.concat([saceld_test_df, test_df.drop(columns=columns_to_scale+['anomaly'])], axis=1)

df_final.head()

,x,y,z,010-000-024-033,010-000-030-096,020-000-032-221,020-000-033-111,label
0,-2.259013,-0.788400,-1.261666,0.0,0.0,1.0,0.0,1.0
1,-2.273531,-0.787290,-1.389401,0.0,1.0,0.0,0.0,1.0
2,-2.273436,-0.791146,-1.378057,1.0,0.0,0.0,0.0,1.0
3,-2.260241,-0.776004,-1.463115,0.0,0.0,1.0,0.0,1.0
4,-2.275263,-0.772365,-1.596010,0.0,1.0,0.0,0.0,1.0


In [10]:
X = df_final.drop(columns = 'label')
y = df_final['label']

X_test = final_test_df.drop(columns = 'label')
y_test = final_test_df['label']

In [11]:
results = {'Method':[], 'Precision':[], 'Recall':[], 'F1-Score':[], 'Supervision':[]}

am = AnomalyMethods()

In [12]:
# 1.KNN
model = NearestNeighbors(n_neighbors=5, radius=0.1, algorithm='auto', leaf_size=30, metric='minkowski', p=2)
knn, knn_y_pred = am.using_knn(X, model)

knn_precision = precision_score(y, knn_y_pred, pos_label=-1)
knn_recall = recall_score(y, knn_y_pred, pos_label=-1)
knn_f1 = f1_score(y, knn_y_pred, pos_label=-1)
print(f'Method: KNN, Precision: {knn_precision}, Recall: {knn_recall}, F1: {knn_f1}')

results['Method'].append('KNN')
results['Precision'].append(knn_precision)
results['Recall'].append(knn_recall)
results['F1-Score'].append(knn_f1)
results['Supervision'].append('Unsupervised')

Method: KNN, Precision: 0.07941001191895113, Recall: 0.08108930473147725, F1: 0.08024087316522394


In [15]:
# 2.Local Outlier Factor (LOF)
lof = LocalOutlierFactor(n_neighbors=30,algorithm='auto',
                         leaf_size=30, metric='minkowski',
                         p=1, metric_params=None, contamination='auto',
                         novelty=False, n_jobs=2
                    )
lof_y_pred = lof.fit_predict(X)

lof_precision = precision_score(y, lof_y_pred, pos_label=-1)
lof_recall = recall_score(y, lof_y_pred, pos_label=-1)
lof_f1 = f1_score(y, lof_y_pred, pos_label=-1)
print(f'Method: LOF, Precision: {lof_precision}, Recall: {lof_recall}, F1: {lof_f1}')

results['Method'].append('Local Outlier Factor')
results['Precision'].append(lof_precision)
results['Recall'].append(lof_recall)
results['F1-Score'].append(lof_f1)
results['Supervision'].append('Unsupervised')

Method: LOF, Precision: 0.004098360655737705, Recall: 0.00030427506465845124, F1: 0.0005664919983005241


In [14]:
# 2.Local Outlier Factor (LOF)
lof = LocalOutlierFactor(n_neighbors=50,algorithm='auto',
                         leaf_size=30, metric='minkowski',
                         p=2, metric_params=None, contamination='auto',
                         novelty=True, n_jobs=2
                    )
lof.fit(X)
lof_y_pred = lof.predict(X_test)

lof_precision = precision_score(y, lof_y_pred, pos_label=-1)
lof_recall = recall_score(y, lof_y_pred, pos_label=-1)
lof_f1 = f1_score(y, lof_y_pred, pos_label=-1)
print(f'Method: LOF, Precision: {lof_precision}, Recall: {lof_recall}, F1: {lof_f1}')

results['Method'].append('Local Outlier Factor')
results['Precision'].append(lof_precision)
results['Recall'].append(lof_recall)
results['F1-Score'].append(lof_f1)
results['Supervision'].append('Unsupervised')

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


ValueError: Found input variables with inconsistent numbers of samples: [134229, 30030]

In [ ]:
# 3.One-Class SVM
ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
ocsvm_y_pred = ocsvm.fit_predict(X)

ocsvm_precision = precision_score(y, ocsvm_y_pred, pos_label=-1)
ocsvm_recall = recall_score(y, ocsvm_y_pred, pos_label=-1)
ocsvm_f1 = f1_score(y, ocsvm_y_pred, pos_label=-1)
print(f'Method: One Class SVM, Precision: {ocsvm_precision}, Recall: {ocsvm_recall}, F1: {ocsvm_f1}')

results['Method'].append('One Class SVM')
results['Precision'].append(ocsvm_precision)
results['Recall'].append(ocsvm_recall)
results['F1-Score'].append(ocsvm_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 4.Isolation Forest
iso = IsolationForest(contamination=0.05, random_state=42)
iso_y_pred = iso.fit_predict(X)

iso_precision = precision_score(y, iso_y_pred, pos_label=-1)
iso_recall = recall_score(y, iso_y_pred, pos_label=-1)
iso_f1 = f1_score(y, iso_y_pred, pos_label=-1)
print(f'Method: Isolation Forest, Precision: {iso_precision}, Recall: {iso_recall}, F1: {iso_f1}')

results['Method'].append('Isolation Forest')
results['Precision'].append(iso_precision)
results['Recall'].append(iso_recall)
results['F1-Score'].append(iso_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 5.Elliptic Envelope
ell = EllipticEnvelope(contamination=0.05)
ell_y_pred = ell.fit_predict(X)

ell_precision = precision_score(y, ell_y_pred, pos_label=-1)
ell_recall = recall_score(y, ell_y_pred, pos_label=-1)
ell_f1 = f1_score(y, ell_y_pred, pos_label=-1)
print(f'Method: Elliptic Envelope, Precision: {ell_precision}, Recall: {ell_recall}, F1: {ell_f1}')

results['Method'].append('Elliptic Envelope')
results['Precision'].append(ell_precision)
results['Recall'].append(ell_recall)
results['F1-Score'].append(ell_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 6.PCA reconstruction error
pca = PCA(n_components=1)
pca, pca_y_pred = am.using_pca(X, pca)

pca_precision = precision_score(y, pca_y_pred, pos_label=-1)
pca_recall = recall_score(y, pca_y_pred, pos_label=-1)
pca_f1 = f1_score(y, pca_y_pred, pos_label=-1)
print(f'Method: PCA, Precision: {pca_precision}, Recall: {pca_recall}, F1: {pca_f1}')

results['Method'].append('PCA')
results['Precision'].append(pca_precision)
results['Recall'].append(pca_recall)
results['F1-Score'].append(pca_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 7.Gaussian Mixture Model
gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42)
gmm.fit(X)
scores = gmm.score_samples(X)
threshold = np.percentile(scores, 5)
gmm_y_pred = np.where(scores < threshold, -1, 1)

gmm_precision = precision_score(y, gmm_y_pred, pos_label=-1)
gmm_recall = recall_score(y, gmm_y_pred, pos_label=-1)
gmm_f1 = f1_score(y, gmm_y_pred, pos_label=-1)
print(f'Method: GMM, Precision: {gmm_precision}, Recall: {gmm_recall}, F1: {gmm_f1}')

results['Method'].append('GMM')
results['Precision'].append(gmm_precision)
results['Recall'].append(gmm_recall)
results['F1-Score'].append(gmm_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 8.K-Means distance method
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X)
distances = np.min(np.abs(X - kmeans.cluster_centers_[labels]), axis=1)
threshold = np.percentile(distances, 95)
kmeans_y_pred = np.where(distances > threshold, -1, 1)

kmeans_precision = precision_score(y, kmeans_y_pred, pos_label=-1)
kmeans_recall = recall_score(y, kmeans_y_pred, pos_label=-1)
kmeans_f1 = f1_score(y, kmeans_y_pred, pos_label=-1)
print(f'Method: K-Means, Precision: {kmeans_precision}, Recall: {kmeans_recall}, F1: {kmeans_f1}')

results['Method'].append('K Means')
results['Precision'].append(kmeans_precision)
results['Recall'].append(kmeans_recall)
results['F1-Score'].append(kmeans_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 9.DBSCAN
dbscan = DBSCAN(eps=0.3, min_samples=75)
labels = dbscan.fit_predict(X)
DBSCAN_y_pred = np.where(labels == -1, -1, 1)

DBSCAN_precision = precision_score(y, DBSCAN_y_pred, pos_label=-1)
DBSCAN_recall = recall_score(y, DBSCAN_y_pred, pos_label=-1)
DBSCAN_f1 = f1_score(y, DBSCAN_y_pred, pos_label=-1)
print(f'Method: DBSCAN, Precision: {DBSCAN_precision}, Recall: {DBSCAN_recall}, F1: {DBSCAN_f1}')

results['Method'].append('DBSCAN')
results['Precision'].append(DBSCAN_precision)
results['Recall'].append(DBSCAN_recall)
results['F1-Score'].append(DBSCAN_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 10.Autoencoder
input_dim = X.shape[1]
autoencoder = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(8, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(input_dim, activation=None)
])
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X, X, epochs=50, batch_size=16, verbose=0)
recon = autoencoder.predict(X)
mse = np.mean(np.square(X - recon), axis=1)
threshold = np.percentile(mse, 95)
auto_y_pred = np.where(mse > threshold, -1, 1)

auto_precision = precision_score(y, auto_y_pred, pos_label=-1)
auto_recall = recall_score(y, auto_y_pred, pos_label=-1)
auto_f1 = f1_score(y, auto_y_pred, pos_label=-1)
print(f'Method: Autoencoder, Precision: {auto_precision}, Recall: {auto_recall}, F1: {auto_f1}')

results['Method'].append('Autoencoder')
results['Precision'].append(auto_precision)
results['Recall'].append(auto_recall)
results['F1-Score'].append(auto_f1)
results['Supervision'].append('Unsupervised')

In [ ]:
# 11.AdaBoost
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X, y)
y_pred_ada = ada.predict(X_test)

ada_precision = precision_score(y_test, y_pred_ada, pos_label=-1)
ada_recall = recall_score(y_test, y_pred_ada, pos_label=-1)
ada_f1 = f1_score(y_test, y_pred_ada, pos_label=-1)
print(f'Method: AdaBoost, Precision: {ada_precision}, Recall: {ada_recall}, F1: {ada_f1}')

results['Method'].append('AdaBoost')
results['Precision'].append(ada_precision)
results['Recall'].append(ada_recall)
results['F1-Score'].append(ada_f1)
results['Supervision'].append('Semi-supervised')

Method: KNN, Precision: 0.07941001191895113, Recall: 0.08108930473147725, F1: 0.08024087316522394
Method: LOF, Precision: 0.08045292014302742, Recall: 0.08215426745778183, F1: 0.08129469326307866
Method: One Class SVM, Precision: 0.07732419547079857, Recall: 0.07895937927886809, F1: 0.0781332329695145


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Method: Isolation Forest, Precision: 0.14421930870083433, Recall: 0.1472691312946904, F1: 0.14572826496048175


/usr/local/lib/python3.11/dist-packages/sklearn/covariance/_robust_covariance.py:184: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-43.360725212556261 > -44.780856538141521). You may want to try with a higher value of support_fraction (current value: 0.503).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/covariance/_robust_covariance.py:184: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-43.215725149967945 > -43.413240594424451). You may want to try with a higher value of support_fraction (current value: 0.503).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/covariance/_robust_covariance.py:184: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-43.298079051534643 > -44.443329519319690). You may want to try with a higher value of support_fraction (current value: 0.503).
  warnings.warn(
/usr/local/lib/p

Method: Elliptic Envelope, Precision: 0.23867699642431467, Recall: 0.24372432679141945, F1: 0.2411742566804667
Method: PCA, Precision: 0.14943384982121574, Recall: 0.1525939449262133, F1: 0.1509973654497554
Method: GMM, Precision: 0.12246722288438618, Recall: 0.12505705157462346, F1: 0.12374858863379752


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Method: K-Means, Precision: 0.034216867469879515, Recall: 0.03240529438612506, F1: 0.033286451007969994
Method: DBSCAN, Precision: 0.07251450290058012, Recall: 0.11029971093868858, F1: 0.08750226298956007


2025-10-26 10:26:02.864479: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


4195/4195 ━━━━━━━━━━━━━━━━━━━━ 4s 990us/step
Method: Autoencoder, Precision: 0.04901668653158522, Recall: 0.05005324813631523, F1: 0.049529544599172
Method: AdaBoost, Precision: 0.2916666666666667, Recall: 0.004347826086956522, F1: 0.008567931456548349


In [55]:
metrics_df = pd.DataFrame(results)
metrics_df

,Method,Precision,Recall,F1-Score,Supervision
0,KNN,0.079410,0.081089,0.080241,Unsupervised
1,Local Outlier Factor,0.080453,0.082154,0.081295,Unsupervised
2,One Class SVM,0.077324,0.078959,0.078133,Unsupervised
3,Isolation Forest,0.144219,0.147269,0.145728,Unsupervised
4,Elliptic Envelope,0.238677,0.243724,0.241174,Unsupervised
5,PCA,0.149434,0.152594,0.150997,Unsupervised
6,GMM,0.122467,0.125057,0.123749,Unsupervised
7,K Means,0.034217,0.032405,0.033286,Unsupervised
8,DBSCAN,0.072515,0.110300,0.087502,Unsupervised
9,Autoencoder,0.049017,0.050053,0.049530,Unsupervised


In [56]:
metrics_df.to_csv('falling_people_results.csv')